# 0. Import Library

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.offsetbox import (OffsetImage,AnnotationBbox)
from matplotlib import transforms
import matplotlib.animation as animation
import time as time

sns.set_theme()

# 2. Data Understanding
Trên thực tế việc khám phá dữ liệu đã được nhóm thực hiện một phần ở notebook Data Preprocessing. Ở notebook này, nhóm chủ yếu tìm hiểu sự phân bố của dữ liệu ở các cột

In [ ]:
df = pd.read_csv('data/kill_match_stats_v2.csv')
df.head()

### 2.3.3. Trung bình khoảng cách theo thời gian

In [ ]:
df[['time', 'dis']].groupby(by = ['time']).mean().plot(figsize = (16, 5), title = 'Average distance by time (included Bluezone)')

In [ ]:
df[df['killed_by'] != 'BlueZone'][['time', 'dis']].groupby(by = ['time']).mean().plot(figsize = (16, 5), title = 'Average distance by time (excluded Bluezone)')

In [ ]:
fig, axes = plt.subplots(nrows = 2, figsize = (16, 11))
fig.subplots_adjust(hspace = 0.4)

data = df[df['type'].isin(df['type'].value_counts()[0:6].index)][['time', 'type', 'dis']].groupby(by = ['time', 'type']).mean().reset_index()
sns.scatterplot(data = data, x = 'time', y = 'dis', hue = 'type', palette = 'Set1_r', ax = axes[0], s = 2, edgecolor = None)
axes[0].set_title('Top 6 upper of Type average distance by time')

data = df[df['type'].isin(df['type'].value_counts()[6:].index)][['time', 'type', 'dis']].groupby(by = ['time', 'type']).mean().reset_index()
sns.scatterplot(data = data, x = 'time', y = 'dis', hue = 'type', palette = 'Set1', ax = axes[1], s = 2, edgecolor = None)
axes[1].set_title('Top 6 lower of Type average distance by time')

### 2.3.4. Phân bố dữ liệu theo tọa độ (2 chiều)
Ở phần trước, nhóm đã biểu diễn phân bố của các cột tọa độ trên 1 chiều (trục ngang). Ở phần này nhóm sẽ biểu diễn phân bố các cột tọa độ theo 2 chiều. 

In [ ]:
fig, axes = plt.subplots(ncols = 2, figsize = (18, 8))
sns.scatterplot(data = df, x = 'kx', y = 'ky', s = 0.1, ax = axes[0])
sns.scatterplot(data = df, x = 'vx', y = 'vy', s = 0.1, ax = axes[1])

axes[0].set_title('Killer position')
axes[1].set_title('Victim position')

In [ ]:
fig, axes = plt.subplots(ncols = 2, figsize = (18, 8), gridspec_kw={'width_ratios': [8, 10]})
fig.subplots_adjust(wspace = 0.1)
img = plt.imread('asset/map/ERANGEL.png')

number_of_bin = 1600

axes[0].imshow(img[::-1, :, 0], extent = [0, 1640, 0, 1640], alpha = 0.35, zorder = 3, cmap = 'gray')
axes[1].imshow(img[::-1, :, 0], extent = [0, 1640, 0, 1640], alpha = 0.35, zorder = 3, cmap = 'gray')

sns.heatmap(
    np.log10(plt.hist2d(df['ky'], df['kx'], bins = number_of_bin, range = [[0, 8000], [0, 8000]])[0] + 1),
    cbar = False, cmap = 'nipy_spectral', ax = axes[0], vmin = 0, vmax = 4)

axes[0].set_xticklabels([])
axes[0].set_yticklabels([])
axes[0].set_title('Killer position')

sns.heatmap(
    np.log10(plt.hist2d(df['vy'], df['vx'], bins = number_of_bin, range = [[0, 8000], [0, 8000]])[0] + 1), 
    cmap = 'nipy_spectral', ax = axes[1], vmin = 0, vmax = 4)

axes[1].set_xticklabels([])
axes[1].set_yticklabels([])
axes[1].set_title('Victim position')

plt.savefig('visualization/heatmap_position.png', dpi = 1200, bbox_inches = 'tight')

In [ ]:
datas = {kill_type : df[df['type'] == kill_type][['vx', 'vy']] for kill_type in df['type'].value_counts().index}
img = plt.imread('asset/map/ERANGEL.png')

In [ ]:
def heatmap_plot(kill_type):
    plt.figure(figsize = (10, 8))

    number_of_bin = 1600

    plt.imshow(img[::-1, :, 0], extent = [0, 1640, 0, 1640], alpha = 0.35, zorder = 3, cmap = 'gray')

    ax =  sns.heatmap(
        np.log(plt.hist2d(datas[kill_type]['vy'], datas[kill_type]['vx'], bins = number_of_bin, range = [[0, 8000], [0, 8000]])[0] + 1), 
        cmap = 'nipy_spectral', vmin = 0, vmax = 8)

    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_title('Victim position by ' + kill_type)
    ax.set_xlim(0, number_of_bin)
    ax.set_ylim(0, number_of_bin)
    ax.invert_yaxis()

    plt.savefig(f'visualization/heatmap_position_by_{kill_type}.png', dpi = 1200, bbox_inches = 'tight')

In [ ]:
for kill_type in df['type'].value_counts().index:
    heatmap_plot(kill_type)

### 2.3.5. Phân bố dữ liệu theo tọa độ và thời gian
Ở phần trên nhóm đã thực hiện trực quan hóa số lượng victim theo tọa độ, ở phần này nhóm tiếp tục phát triển thêm một chiều thời gian để thể hiện sự phân bố của victim trên bản đồ trong suốt trận đấu.

In [ ]:
img = plt.imread('asset/map/ERANGEL.png')

In [ ]:
%matplotlib inline

def ax_prunt(ax):
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_xlim(0, number_of_bin)
    ax.set_ylim(0, number_of_bin)
    ax.invert_yaxis()

def init():
    sns.heatmap(np.zeros((number_of_bin, number_of_bin)), vmin = 0, vmax = 8, cmap = 'nipy_spectral', cbar = False)
    
    ax_prunt(ax)
def animate(i):
    data = df[df['time'].isin(range(i, i + 50))][['vx', 'vy']]
    data = np.log(plt.hist2d(data['vy'], data['vx'], bins = number_of_bin, range = [[0, 8000], [0, 8000]])[0] + 1)
    sns.heatmap(data, cmap = 'nipy_spectral', vmin = 0, vmax = 8, cbar = False)

    ax.set_title(f'{i}s - {i + 50}s')
    ax_prunt(ax)
    print(i)

fig, ax = plt.subplots(figsize = (10, 8))
number_of_bin = 1600
scale = number_of_bin / 1600 * 1640

plt.imshow(img[::-1, :, 0], extent = [0, scale, 0, scale], alpha = 0.35, zorder = 3, cmap = 'gray')
ax_prunt(ax)
sns.heatmap(np.zeros((number_of_bin, number_of_bin)), vmin = 0, vmax = 8, cmap = 'nipy_spectral')

anim = animation.FuncAnimation(fig, func = animate, init_func = init, interval = 20, frames = range(1300, df['time'].max() - 40, 20), repeat = False)

start_time = time.time()

writer = animation.FFMpegWriter(fps=10)
anim.save('test3.mp4', writer=writer)

print(f'{time.time() - start_time:.2f} sec for saving video')  